In [1]:
from bs4 import BeautifulSoup
import requests
import numpy as np
import re
import pandas as pd
import datetime
import os
from os.path import isfile, join

In [2]:
start_year="2022"
end_year="2023"

url="https://sinqlair.carm.es/calidadaire/obtener_datos.aspx?tipo=tablaRedVigilancia"

data_dates={
    'tipoConsulta': "medias_diarias",
    'estacionesSelec': "e_sanbasilio_San Basilio",
    'contaminantesSelec': "NO2-NOX*NOx-O3-PM10-PM25",
    'periodoConsulta': "sel_rango",
    'tipo_dato': "grid",
    'fechaInicioConsulta': "01/01/"+start_year,
    'fechaFinConsulta': "01/01/"+end_year
}

headers= {
    'Host': "sinqlair.carm.es",
    'User-Agent': "Mozilla/5.0 Gecko/20100101 Firefox/70.0",
    'Accept': "*/*",
    'Accept-Language': "en-US,en;q=0.5",
    'Accept-Encoding': "gzip, deflate, br",
    'X-Requested-With': "XMLHttpRequest",
    'Content-Type': "application/x-www-form-urlencoded; charset=UTF-82",
    'Content-Length': "114",
    'Origin': "https://sinqlair.carm.es",
    'DNT': "1",
    'Connection': "keep-alive",
    'Referer': "https://sinqlair.carm.es/calidadaire/estaciones/estacion.aspx?San%20Basilio",
    'Cookie': "ASP.NET_SessionId=qhnag2eedjorexs450mudas1"
}

def tidy_data(data):
    data.columns = ['fecha', 'station', 'limit', 'NO2', 'NOX','O3','PM10','PM25']

    data['FULL_DATE'] = data['fecha'].apply(str_date)
    data['DATE'] = data['FULL_DATE'].apply(date_date)
    data['DAY'] = data['FULL_DATE'].apply(day_date)
    data['MONTH'] = data['FULL_DATE'].apply(month_date)
    data['YEAR'] = data['FULL_DATE'].apply(year_date)
    data['HOUR'] = data['FULL_DATE'].apply(hour_date)
    data['WEEKDAY'] = data['FULL_DATE'].apply(weekday_date)
    data['WEEKEND'] = data['FULL_DATE'].apply(weekend_date)
    data['PM10_Q'] = data['PM10'].apply(pm_10q)
    data['PM25_Q'] = data['PM25'].apply(pm_25q)
    data['QUALITY'] = data['PM10'].apply(quality)
    
    data = data[['DATE','DAY','MONTH','YEAR','HOUR','WEEKDAY','WEEKEND','O3','NO2','NOX','PM10','PM10_Q','PM25','PM25_Q','QUALITY']]
    data = data.replace('---', np.nan)
    return data

def str_date(date):
    return datetime.datetime.strptime(date,"%d/%m/%Y %H:%M:%S")

def date_date(date):
    return date.strftime("%d/%m/%Y")

def day_date(date):
    return date.day

def month_date(date):
    return date.month

def year_date(date):
    return date.year

def hour_date(date):
    return date.hour

def weekday_date(date):
    return date.isoweekday()

def weekend_date(date):
    return "Weekend" if date.isoweekday()> 5 else "Weekday"

def pm_10q(pm10):
    pm10 = pd.to_numeric(pm10, errors='coerce')
    if (pm10 <= 20): 
        return "Buena"
    if (pm10 >20 and pm10 <=40 ):
        return "Razonablemente buena"
    if (pm10 >40 and pm10 <=50 ):
        return "Regular"
    if (pm10 >50 and pm10 <=100 ):
        return "Desfavorable"
    if (pm10 >100 and pm10 <=150 ):
        return "Muy desfavorable"
    if (pm10 >150):
        return "Extremandamente desfavorable"

def pm_25q(pm25):
    pm25 = pd.to_numeric(pm25, errors='coerce')
    if (pm25 <= 10): 
        return "Buena"
    if (pm25 >10 and pm25 <=20 ):
        return "Razonablemente buena"
    if (pm25 >20 and pm25 <=25 ):
        return "Regular"
    if (pm25 >25 and pm25 <=50 ):
        return "Desfavorable"
    if (pm25 >50 and pm25 <=75 ):
        return "Muy desfavorable"
    if (pm25 >75):
        return "Extremandamente desfavorable"

def quality(pm10):
    pm10 = pd.to_numeric(pm10, errors='coerce')
    if (pm10 <= 50): 
        return "GOOD"
    if (pm10 >50 ):
        return "BAD"

    
r = requests.post(url, data=data_dates, headers=headers)
data = r.text.replace("ï»¿","")
data = data.replace("\r\n","")

data = pd.read_json(data)
data = tidy_data(data)
data.to_csv("../data/medias_diarias-all.csv", index=None, header=True)


In [3]:
r = requests.post(url, data=data_dates, headers=headers)
data = r.text.replace("ï»¿","")
data = data.replace("\r\n","")

data = pd.read_json(data)
data = tidy_data(data)
data.to_csv("../data/mean-daily-"+start_year+".csv", index=None, header=True)

In [4]:
data

,DATE,DAY,MONTH,YEAR,HOUR,WEEKDAY,WEEKEND,O3,NO2,NOX,PM10,PM10_Q,PM25,PM25_Q,QUALITY
0,01/01/2022,1,1,2022,0,6,Weekend,16,22,45,83,Desfavorable,48,Desfavorable,BAD
1,02/01/2022,2,1,2022,0,7,Weekend,12,21,39,42,Regular,20,Razonablemente buena,GOOD
2,03/01/2022,3,1,2022,0,1,Weekday,9,25,64,41,Regular,22,Regular,GOOD
3,04/01/2022,4,1,2022,0,2,Weekday,19,22,49,33,Razonablemente buena,16,Razonablemente buena,GOOD
4,05/01/2022,5,1,2022,0,3,Weekday,55,17,29,13,Buena,6,Buena,GOOD
5,06/01/2022,6,1,2022,0,4,Weekday,39,16,29,15,Buena,10,Buena,GOOD
6,07/01/2022,7,1,2022,0,5,Weekday,25,21,44,23,Razonablemente buena,17,Razonablemente buena,GOOD
7,08/01/2022,8,1,2022,0,6,Weekend,35,20,36,27,Razonablemente buena,16,Razonablemente buena,GOOD
8,09/01/2022,9,1,2022,0,7,Weekend,45,16,26,23,Razonablemente buena,10,Buena,GOOD
9,10/01/2022,10,1,2022,0,1,Weekday,44,14,26,13,Buena,6,Buena,GOOD
